# Data Wrangling

of Project 'Demography behind household sizes in Switzerland and its change in time'

In [1]:
%matplotlib inline
import os
import pandas as pd
import json
import folium
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
data_folder = './data/'

## 1) Load and Prepare Data

In [21]:
hh_sizes_2017 = pd.read_excel(data_folder + 'HH_sizes_commune.xlsx', skiprows=3)

In [71]:
#read whole excel sheet
hh_sizes = pd.read_excel(data_folder + 'HH_sizes_commune.xlsx', skiprows=3, skipfooter=10, sheet_name = None, usecols=list(range(9)))

In [85]:
#access sheets or columns per:

#rename headers and index to commune
header = ['Commune', 'Total: 1', '1-person households','2-person households', '3-person households', '4-person households','5-person households', '6-person or larger households', 'not plausible hh']
years = ['2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010']

for y in years:
    hh_sizes[y].columns = header
    hh_sizes[y].set_index('Commune')


In [86]:
hh_sizes['2015']

,Commune,Total: 1,1-person households,2-person households,3-person households,4-person households,5-person households,6-person or larger households,not plausible hh
0,Switzerland,8156286,1275667,2376720,1446417,1857428,789900,410154,< 0.5
1,0001 Aeugst am Albis,1970,237,534,336,520,240,103,< 0.5
2,0002 Affoltern am Albis,11494,1646,3420,1932,2492,1235,769,< 0.5
3,0003 Bonstetten,5315,641,1550,918,1564,470,172,< 0.5
4,0004 Hausen am Albis,3450,388,1086,558,936,300,182,< 0.5
5,0005 Hedingen,3642,395,1064,594,1104,335,150,< 0.5
6,0006 Kappel am Albis,1031,119,300,105,276,170,61,0
7,0007 Knonau,2089,207,576,381,568,220,137,0
8,0008 Maschwanden,638,79,170,105,184,75,25,0
9,0009 Mettmenstetten,4610,472,1454,810,1204,495,175,1
